In [ ]:
# install necesary packages
%pip install nbimporter
%pip install nbformat

%pip install pandas
%pip install numpy
%pip install langchain langchain-community openai

In [ ]:
from nbimporter import NotebookLoader
import sys
sys.meta_path.append(NotebookLoader())

# Lang chain imports
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI  
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

# Import features (TODO: Add features as we develop)
from summaries import get_summaries
from outliers import detect_outliers

In [ ]:
def main(user_query, df):
    log = [] # To be implemented later

    # LLM Prompt
    prompt = f"""
    User wants: "{user_query}"
    Dataset: {df.shape[0]} rows, {df.shape[1]} columns
    Which feature(s) should handle this request?

    Available features:
    summaries: get_summary(user_query, df)
    imputation: impute_missing(user_query, df)
    outliers: detect_outliers(user_query, df)

    Generate Python code that calls the needed functions in order.
    Each function takes (user_query, df) and returns modified df.
    Store final result in 'df'.
    
    Example: df = impute_missing(user_query, df)
    Or multistep: df = impute_missing(user_query, df); df = get_summary(user_query, df)
    
    Return only executable Python code, no explanations.
    """

    # Call LLM
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo") # gpt-3.5 for development
    response = llm.invoke([HumanMessage(content=prompt)])
    generated_code = response.content.strip()

    # Clean up Code (TODO)

    # Execute Code (Split into separate function later maybe)
    try:
        exec_env = {
            'df': df.copy(),
            'user_query': user_query,
            'get_summary': get_summary,
            'impute_missing': impute_missing, # Example
            'transform_data': transform_data, # Example
            'detect_outliers': detect_outliers, # Example
            'pd': pd,
            'np': np
        }
        
        exec(generated_code, exec_env)
        return exec_env['df']
    except Exception as e:
        print(f"Error: {e}")
        return df
